<a href="https://colab.research.google.com/github/diegarza/Regresion_Logistica/blob/main/Actividad7_A01705984.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Importación del documento**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('cuentas_credicel.csv', encoding = 'latin')

##**Tratamiento de nulos y outliers**

In [118]:
########## Valores nulos #############

df.dropna(inplace=True)

# Columna 'limite_credito' debe corregirse
# Verifica y reemplaza valores no numéricos con la mediana

import re

def eliminar_letras(cadena):
  if isinstance(cadena, (int, float)):
    return cadena

  cadena_numerica = ""

  for caracter in str(cadena):
    if caracter.isdigit() or caracter == '.':
      cadena_numerica += caracter

  try:
    valor_numerico = float(cadena_numerica)
  except ValueError:
    valor_numerico = 0.0

  return valor_numerico

df['limite_credito'] = df['limite_credito'].astype(str).apply(eliminar_letras)

df['limite_credito'] = df['limite_credito'].apply(eliminar_letras)

######### Elimincación de outliers ############

for col in df.columns:
  if col != 'folio' and col != 'folio_solicitud':
    if df[col].dtypes == 'int64' or df[col].dtypes == 'float64':
      Q1 = df[col].quantile(0.25)
      Q3 = df[col].quantile(0.75)
      IQR = Q3 - Q1
      lower_bound = Q1 - 1.5 * IQR
      upper_bound = Q3 + 1.5 * IQR
      df.loc[df[col] < lower_bound, col] = df[col].mean()
      df.loc[df[col] > upper_bound, col] = df[col].mean()

df.isnull().sum().sum()

0

In [119]:
df = df.drop(columns = [i for i in df.columns if df[i].dtypes == 'object'])

##**Todas las variables como X y score_buro como y**

In [120]:
resultados = pd.DataFrame(columns=['Variable', 'Precisión', 'Exactitud', 'Puntaje F1'])

# Transformar columna score a valor categórico

df_1 = df.copy()

percentil_50 = df['score_buro'].quantile(0.5)

# Crea una función para categorizar los valores
def categorizar_valor(valor):
    if valor <= percentil_50:
        return 'bajo'
    else:
        return 'alto'

# Aplica la función para categorizar la columna
df_1['score_buro'] = df_1['score_buro'].apply(categorizar_valor)

In [121]:
y = df_1['score_buro']
X = df_1[['precio',
        'costo_total',
        'enganche',
        'monto_financiado',
        'monto_accesorios',
        'pagos_realizados',
        'reautorizacion',
        'limite_credito',
        'descuento',
        'edad_cliente']]

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state =None)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [123]:
y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="binary", pos_label="alto")
exactitud = accuracy_score(y_test, y_pred)
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label="alto")

var = 'Todas'

resultados = resultados.append({'Variable': var,
                                  'Precisión': precision,
                                  'Exactitud': exactitud,
                                  'Puntaje F1': puntajef1},
                                 ignore_index=True)

print('Matriz de Confusión:')
print(matriz)
print('\nPrecisión del modelo:')
print(precision)
print('\nExactitud del modelo:')
print(exactitud)
print('\nPuntaje F1 del modelo:')
print(puntajef1)

Matriz de Confusión:
[[1486  616]
 [ 257 1928]]

Precisión del modelo:
0.8525530694205393

Exactitud del modelo:
0.7963610916724982

Puntaje F1 del modelo:
0.7729518855656696


<ipython-input-123-ca4a91366669>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultados = resultados.append({'Variable': var,


##**Cada una de las variables seleccionadas comparada con score_buro**

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

cont = 1


for i in X.columns:

  escalar = StandardScaler()
  X_train_i = X_train[[i]]
  X_test_i = X_test[[i]]
  X_train_i = escalar.fit_transform(X_train_i)
  X_test_i = escalar.transform(X_test_i)
  algoritmo = LogisticRegression()
  algoritmo.fit(X_train_i, y_train)
  y_pred = algoritmo.predict(X_test_i)

  matriz = confusion_matrix(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average="binary", pos_label="alto")
  exactitud = accuracy_score(y_test, y_pred)
  puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label="alto")


  resultados = resultados.append({'Variable': i,
                                  'Precisión': precision,
                                  'Exactitud': exactitud,
                                  'Puntaje F1': puntajef1},
                                 ignore_index=True)

  print('________________________________')
  print(f'\n {cont}. Variable {i} con score_buro')
  print('\nMatriz de Confusión:')
  print(matriz)
  print('\nPrecisión del modelo:')
  print(precision)
  print('\nExactitud del modelo:')
  print(exactitud)
  print('\nPuntaje F1 del modelo:')
  print(puntajef1)
  cont += 1

<ipython-input-124-c3284906aa1d>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultados = resultados.append({'Variable': i,


________________________________

 1. Variable precio con score_buro

Matriz de Confusión:
[[ 816 1370]
 [ 713 1388]]

Precisión del modelo:
0.5336821451929366

Exactitud del modelo:
0.5141124329367857

Puntaje F1 del modelo:
0.439300134589502


<ipython-input-124-c3284906aa1d>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultados = resultados.append({'Variable': i,


________________________________

 2. Variable costo_total con score_buro

Matriz de Confusión:
[[1100 1086]
 [ 821 1280]]

Precisión del modelo:
0.5726184279021344

Exactitud del modelo:
0.5551667832983438

Puntaje F1 del modelo:
0.5356708059410762


<ipython-input-124-c3284906aa1d>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultados = resultados.append({'Variable': i,


...

<ipython-input-124-c3284906aa1d>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultados = resultados.append({'Variable': i,


________________________________

 8. Variable limite_credito con score_buro

Matriz de Confusión:
[[1617  569]
 [ 234 1867]]

Precisión del modelo:
0.873581847649919

Exactitud del modelo:
0.8126895264753907

Puntaje F1 del modelo:
0.8010899182561309


<ipython-input-124-c3284906aa1d>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultados = resultados.append({'Variable': i,


________________________________

 9. Variable descuento con score_buro

Matriz de Confusión:
[[ 674 1512]
 [ 605 1496]]

Precisión del modelo:
0.5269741985926505

Exactitud del modelo:
0.5061814788896665

Puntaje F1 del modelo:
0.389033189033189
________________________________

 10. Variable edad_cliente con score_buro

Matriz de Confusión:
[[ 995 1191]
 [ 839 1262]]

Precisión del modelo:
0.5425299890948746

Exactitud del modelo:
0.5264753907161185

Puntaje F1 del modelo:
0.49502487562189057


<ipython-input-124-c3284906aa1d>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultados = resultados.append({'Variable': i,


In [125]:
resultados

,Variable,Precisión,Exactitud,Puntaje F1
0,Todas,0.852553,0.796361,0.772952
1,precio,0.533682,0.514112,0.439300
2,costo_total,0.572618,0.555167,0.535671
3,enganche,0.562414,0.553767,0.562543
4,monto_financiado,0.559116,0.540005,0.506507
5,monto_accesorios,0.000000,0.490086,0.000000
6,pagos_realizados,0.538509,0.519244,0.458913
7,reautorizacion,0.000000,0.490086,0.000000
8,limite_credito,0.873582,0.812690,0.801090
9,descuento,0.526974,0.506181,0.389033
